In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append(r'.\Pycharm')
from Captcha import gen_captcha,numbers

def text2vec(text):#text变为one hot coding
    vector = np.zeros(40)
    for i in range(len(text)):
        idx = i*10+ord(text[i])-48#ACSII转int
        vector[idx] = 1
    return vector

def gen_batch(size=100):#生成一批数据
    batch_x = np.zeros((size,9600))
    batch_y = np.zeros((size,40))
    for i in range(size):
        text,image,data = gen_captcha()
        batch_x[i,:] = np.mean(image,-1).flatten()/255#rgb转为灰度值，再归一化
        batch_y[i,:] = text2vec(text)#text变为one hot coding
    return batch_x,batch_y
    
with tf.name_scope('input') as scope:#name_scope可以让变量有相同的命名，只是限于tf.Variable的变量，可以看做程序里的一个单元
    x = tf.placeholder(tf.float32,[None,60*160],name='x')#N张图片，每张图片像素60*160，放入灰度图
    y_ = tf.placeholder(tf.float32,[None,4*10],name='y_')#one hot coding 6;[0,0,0,0,0,0,1,0,0,0],每个y_ 4个数字，每个数字用1*10向量表示
    
with tf.name_scope('net') as scope:
    x_ = tf.reshape(x,shape=[-1,60,160,1])
    #卷积神经网络
    with tf.name_scope('conv1') as scope:
        w_c1 = tf.Variable(0.01*tf.random_normal([3,3,1,1]))#*0.01因为不乘的话数字太大，收敛慢
        #w_c1 = tf.Variable(0.01*tf.random_normal([3,3,1,32]))
        b_c1 = tf.Variable(0.1*tf.random_normal([1]))
        #b_c1 = tf.Variable(0.1*tf.random_normal([32]))
        #卷积
        conv1 = tf.nn.conv2d(x_,w_c1,strides=[1,1,1,1],padding='SAME')
        image = tf.reshape(conv1,shape=[-1,60,160,1])
        tf.summary.image('image1-1',image)
        #激活
        conv1 = tf.nn.relu(tf.nn.bias_add(conv1,b_c1))
        image = tf.reshape(conv1,shape=[-1,60,160,1])
        tf.summary.image('image1-2',image)
        #池化
        conv1 = tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        image = tf.reshape(conv1,shape=[-1,30,80,1])
        tf.summary.image('image1-3',image)
     #Dense
    with tf.name_scope('dense') as scope:
        w_d  = tf.Variable(tf.random_normal([30*80,1024]))
        #w_d  = tf.Variable(tf.random_normal([32*30*80,1024]))
        b_d = tf.Variable(tf.random_normal([1024]))
        dense = tf.reshape(conv1,[-1,w_d.get_shape().as_list()[0]])
        dense = tf.nn.relu(tf.add(tf.matmul(dense,w_d),b_d))
        dense = tf.nn.dropout(dense, 0.75)
    #输出
    with tf.name_scope('output') as scope:
        w = tf.Variable(0.01*tf.random_normal([1024,40]),name = 'w')
        b = tf.Variable(0.1*tf.random_normal([40]),name = 'b')
        y = tf.matmul(dense,w)+b
    
with tf.name_scope('training') as scope:
    #loss = tf.losses.mean_squared_error(y_, y)
    loss = tf.losses.sigmoid_cross_entropy(y_,y)
    tf.summary.scalar('loss',loss)#记录并统计loss
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    
with tf.name_scope('predict') as scope:
    p = tf.argmax(tf.reshape(y,[-1,4,10]),2) #在第二个维度将one hot coding转为十进制数，投票的方式
    label = tf.argmax(tf.reshape(y_,[-1,4,10]),2)
    pred = tf.equal(p,label)#判断预测和真实值是否相等
    accuracy = tf.reduce_mean(tf.cast(pred,tf.float32))
    tf.summary.scalar('accuracy',accuracy)
 
       
with tf.Session() as sess:
    merged = tf.summary.merge_all()    
    tf.global_variables_initializer().run()
    writer = tf.summary.FileWriter('./log/demo18',sess.graph)
    
    
    step=0
    for i in range(5000):
        #训练
        x_train,y_train = gen_batch(64)
        _,loss_ = sess.run([optimizer,loss],feed_dict={x:x_train,y_:y_train})
        print(step,loss_)
        #每100批测试一次准确率
        if step%50 == 0:
            x_test,y_test = gen_batch(50)
            #acc = sess.run([accuracy], feed_dict={x:x_test, y_:y_test})
            summary,acc = sess.run([merged,accuracy],feed_dict={x:x_test,y_:y_test})
            writer.add_summary(summary,step)
            print(acc)
        step = step + 1

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


0 0.727801
0.15
1 0.42530435
2 0.3384133
3 0.35763735
4 0.38302475
5 0.37854943
6 0.3603
7 0.34996805
8 0.3453765
9 0.34481463
10 0.34402245
11 0.33804312
12 0.3439949
13 0.33690542
14 0.33366284
15 0.3293373
16 0.32944846
17 0.33011702
18 0.32700247
19 0.32927775
20 0.33178884
21 0.33322126
22 0.32992196
23 0.33243045
24 0.3293614
25 0.33031684
26 0.33135873
27 0.32778454
28 0.32723168
29 0.3262879
30 0.32488436
31 0.32766095
32 0.3260027
33 0.3276744
34 0.32784864
35 0.32560557
36 0.32582775
37 0.32773328
38 0.3261251
39 0.3269537
40 0.3230862
41 0.32652703
42 0.32542115
43 0.3233245
44 0.32564515
45 0.32276988
46 0.3237415
47 0.32237226
48 0.32458833
49 0.32245463
50 0.32598618
0.13
51 0.3262863
52 0.327913
53 0.32359743
54 0.3255423
55 0.3233173
56 0.32340285
57 0.32777578
58 0.32643366
59 0.32302198
60 0.32381886
61 0.32134503
62 0.32373136
63 0.32565144
64 0.32183146
65 0.3204492
66 0.32220456
67 0.3205306
68 0.32432827
69 0.3253492
70 0.32192492
71 0.32185498
72 0.320391
73 0.32

562 0.24807315
563 0.24984343
564 0.24585967
565 0.26456413
566 0.24991925
567 0.24619272
568 0.26594836
569 0.26005825
570 0.24849734
571 0.25041658
572 0.24837372
573 0.25355375
574 0.25782385
575 0.24845442
576 0.24964003
577 0.25590342
578 0.24411964
579 0.23968573
580 0.23702757
581 0.24165031
582 0.23765321
583 0.25948977
584 0.25826973
585 0.23669195
586 0.24094729
587 0.2552197
588 0.24654892
589 0.25514883
590 0.24208388
591 0.24058516
592 0.24575749
593 0.25998205
594 0.2468262
595 0.24564996
596 0.2425712
597 0.23881026
598 0.23979643
599 0.25579718
600 0.24034627
0.43
601 0.25535142
602 0.25623944
603 0.24862532
604 0.25070137
605 0.24941358
606 0.23766363
607 0.24273007
608 0.23721854
609 0.25661477
610 0.2646664
611 0.23664758
612 0.24321583
613 0.24129629
614 0.2422221
615 0.23697853
616 0.2565084
617 0.24502175
618 0.24705772
619 0.24590854
620 0.25207865
621 0.24592546
622 0.2422804
623 0.24200244
624 0.24883814
625 0.24859324
626 0.24280913
627 0.24453345
628 0.246486

KeyboardInterrupt: 